In [1]:
import numpy as np
import argparse
import time
import os
import collections
import json
import queue
import time

from utils.data_utils import load_dataset_numpy

import scipy.spatial.distance

from scipy.sparse import csr_matrix, coo_matrix
from scipy.sparse.csgraph import maximum_flow
from utils.flow import _make_edge_pointers

from cvxopt import solvers, matrix, spdiag, log, mul, sparse, spmatrix

In [2]:
def minll(G,h,p):
    m,v_in=G.size
    def F(x=None,z=None):
        if x is None:
            return 0, matrix(1.0,(v,1))
        if min(x)<=0.0:
            return None
        f = -sum(mul(p,log(x)))
        Df = mul(p.T,-(x**-1).T)
        if z is None:
            return f,Df
        # Fix the Hessian
        H = spdiag(z[0]*mul(p,x**-2))
        return f,Df,H
    return solvers.cp(F,G=G,h=h)

In [3]:
def find_remaining_cap_edges(edge_ptr,capacities,heads,tails, source, sink):
    ITYPE = np.int32
    n_verts = edge_ptr.shape[0] - 1
    n_edges = capacities.shape[0]
    ITYPE_MAX = np.iinfo(ITYPE).max

    # Our result array will keep track of the flow along each edge
    flow = np.zeros(n_edges, dtype=ITYPE)

    # Create a circular queue for breadth-first search. Elements are
    # popped dequeued at index start and queued at index end.
    q = np.empty(n_verts, dtype=ITYPE)

    # Create an array indexing predecessor edges
    pred_edge = np.empty(n_verts, dtype=ITYPE)

    # While augmenting paths from source to sink exist
    for k in range(n_verts):
        pred_edge[k] = -1
    path_edges = []
    # Reset queue to consist only of source
    q[0] = source
    start = 0
    end = 1
    # While we have not found a path, and queue is not empty
    path_found = False
    while start != end and not path_found:
        # Pop queue
        cur = q[start]
        start += 1
        if start == n_verts:
            start = 0
        # Loop over all edges from the current vertex
        for e in range(edge_ptr[cur], edge_ptr[cur + 1]):
            t = heads[e]
            if pred_edge[t] == -1 and t != source and\
                    capacities[e] > flow[e]:
                pred_edge[t] = e
                path_edges.append((cur,t))
                if t == sink:
                    path_found = True
                    break
                # Push to queue
                q[end] = t
                end += 1
                if end == n_verts:
                    end = 0
    return path_edges


In [4]:
def create_graph_rep(edge_matrix,n_1,n_2):
    graph_rep = []
    for i in range(n_1+n_2+2):
        graph_rep.append([])
        if i==0:
            #source
            for j in range(n_1+n_2+2):
                if j==0:
                    graph_rep[i].append(0)
                elif 1<=j<=n_1:
                    graph_rep[i].append(1)
                elif n_1<j<=n_1+n_2+1:
                    graph_rep[i].append(0)
        elif 1<=i<=n_1:
            # LHS vertices
            for j in range(n_1+n_2+2):
                if j<=n_1:
                    graph_rep[i].append(0)
                elif n_1<j<=n_1+n_2:
                    if edge_matrix[i-1,j-n_1-1]:
                        graph_rep[i].append(1)
                    else:
                        graph_rep[i].append(0)
                elif n_1+n_2<j:
                    graph_rep[i].append(0)
        elif n_1<i<=n_1+n_2:
            #RHS vertices
            for j in range(n_1+n_2+2):
                if j<=n_1+n_2:
                    graph_rep[i].append(0)
                elif j>n_1+n_2:
                    graph_rep[i].append(1)
        elif i==n_1+n_2+1:
            #Sink
            for j in range(n_1+n_2+2):
                graph_rep[i].append(0)

    graph_rep_array=np.array(graph_rep)

    return graph_rep_array

In [5]:
def set_classifier_prob_full_flow(top_level_vertices,w_1_curr,w_2_curr):
    for item in top_level_vertices:
        if item !=0 and item != sink_idx:
            classifier_probs[item-1,0]=w_1_curr/(w_1_curr+w_2_curr)
            classifier_probs[item-1,1]=w_2_curr/(w_1_curr+w_2_curr)

In [6]:
def set_classifier_prob_no_flow(top_level_vertices):
    for item in top_level_vertices:
        if item !=0 and item != sink_idx:
            if item<=n_1:
                classifier_probs[item-1,0]=1
                classifier_probs[item-1,1]=0
            elif item>n_1:
                classifier_probs[item-1,0]=0
                classifier_probs[item-1,1]=1

In [7]:
def graph_rescale(graph_rep_curr,top_level_indices,weights):
    class_1_vertices=top_level_indices[( top_level_indices> 0) & (top_level_indices<=n_1)]
    class_1_weights=weights[class_1_vertices]
    w_1_curr=np.sum(class_1_weights)
    class_2_vertices=top_level_indices[ (top_level_indices>n_1) & (top_level_indices<n_1+n_2)]
    class_2_weights=weights[class_2_vertices]
    w_2_curr=np.sum(class_2_weights)
    n_1_curr=len(class_1_vertices)
    n_2_curr=len(class_2_vertices)
    #n_1_curr=len(np.where(top_level_indices<=n_1)[0])-1
    #n_2_curr=len(np.where(top_level_indices>n_1)[0])-1
    # source rescale
    # print(graph_rep_curr[0])
    #graph_rep_curr[0,:]=graph_rep_curr[0,:]/n_2
    graph_rep_curr[0,:]= (graph_rep_curr[0,:]*w_2_curr)
    # print(graph_rep_curr[0])
    # bipartite graph edge scale
    #graph_rep_curr[1:n_1_curr+1,:]=graph_rep_curr[1:n_1_curr+1,:]/(n_1*n_2)
    graph_rep_curr[1:n_1_curr+1,:]=(graph_rep_curr[1:n_1_curr+1,:]*(w_1_curr*w_2_curr))
    # sink edges rescale
    #graph_rep_curr[n_1_curr+1:,:]=graph_rep_curr[n_1_curr+1:,:]/n_1
    graph_rep_curr[n_1_curr+1:,:]=(graph_rep_curr[n_1_curr+1:,:]*w_1_curr)
    return graph_rep_curr,w_1_curr,w_2_curr

In [8]:
def find_flow_and_split(top_level_indices,weights):
    top_level_indices_1=None
    top_level_indices_2=None
    #Create subgraph from index array provided
    graph_rep_curr = graph_rep_array[top_level_indices]
    graph_rep_curr = graph_rep_curr[:,top_level_indices]
    graph_rep_curr,w_1_curr,w_2_curr = graph_rescale(graph_rep_curr,top_level_indices,weights)
    graph_curr=csr_matrix(graph_rep_curr)
    flow_curr = maximum_flow(graph_curr,0,len(top_level_indices)-1)
    # Checking if full flow occurred, so no need to split
    if flow_curr.flow_value==w_1_curr*w_2_curr:
        set_classifier_prob_full_flow(top_level_indices,w_1_curr,w_2_curr)
        return top_level_indices_1,top_level_indices_2, flow_curr
    elif flow_curr.flow_value==0:
        set_classifier_prob_no_flow(top_level_indices)
        return top_level_indices_1,top_level_indices_2, flow_curr
    # Finding remaining capacity edges
    remainder_array = graph_curr-flow_curr.residual

    rev_edge_ptr, tails = _make_edge_pointers(remainder_array)

    edge_ptr=remainder_array.indptr
    capacities=remainder_array.data
    heads=remainder_array.indices

    edge_list_curr = find_remaining_cap_edges(edge_ptr,capacities,heads,tails,0,len(top_level_indices)-1)

#     print(edge_list_curr)
    gz_idx = []
    for item in edge_list_curr:
        gz_idx.append(item[0])
        gz_idx.append(item[1])
    if len(gz_idx)>0:
        gz_idx=np.array(gz_idx)
        gz_idx_unique=np.unique(gz_idx)
        top_level_gz_idx=top_level_indices[gz_idx_unique]
        top_level_gz_idx=np.insert(top_level_gz_idx,len(top_level_gz_idx),sink_idx)
        top_level_indices_1=top_level_gz_idx
    else:
        top_level_gz_idx=np.array([0,sink_idx])
    # Indices without flow
    top_level_z_idx=np.setdiff1d(top_level_indices,top_level_gz_idx)
    if len(top_level_z_idx)>0:
        # Add source and sink back to zero flow idx array
        top_level_z_idx=np.insert(top_level_z_idx,0,0)
        top_level_z_idx=np.insert(top_level_z_idx,len(top_level_z_idx),sink_idx)
        top_level_indices_2=top_level_z_idx
    
    return top_level_indices_1,top_level_indices_2, flow_curr

In [14]:
parser = argparse.ArgumentParser()


In [15]:
parser.add_argument("--dataset_in", default='MNIST',
                    help="dataset to be used")
parser.add_argument("--norm", default='l2',
                    help="norm to be used")
parser.add_argument('--num_samples', type=int, default=None)
parser.add_argument('--n_classes', type=int, default=2)
parser.add_argument('--eps', type=float, default=None)
parser.add_argument('--approx_only', dest='approx_only', action='store_true')
parser.add_argument('--use_test', dest='use_test', action='store_true')
parser.add_argument('--track_hard', dest='track_hard', action='store_true')
parser.add_argument('--use_full', dest='use_full', action='store_true')
parser.add_argument('--run_generic', dest='run_generic', action='store_true')
parser.add_argument('--new_marking_strat', type=str, default=None)
parser.add_argument('--num_reps', type=int, default=2)
parser.add_argument('--class_1', type=int, default=3)
parser.add_argument('--class_2', type=int, default=7)

_StoreAction(option_strings=['--class_2'], dest='class_2', nargs=None, const=None, default=7, type=<class 'int'>, choices=None, help=None, metavar=None)

In [16]:
args = parser.parse_args("--dataset_in=MNIST --num_samples=500 --use_full --eps=4".split())

In [17]:
args

Namespace(approx_only=False, class_1=3, class_2=7, dataset_in='MNIST', eps=4.0, n_classes=2, new_marking_strat=None, norm='l2', num_reps=2, num_samples=500, run_generic=False, track_hard=False, use_full=True, use_test=False)

In [18]:
train_data, test_data, data_details = load_dataset_numpy(args, data_dir='data',
                                                        training_time=False)

In [19]:
train_data

Dataset MNIST
    Number of datapoints: 1000
    Root location: data
    Split: Train

In [20]:
test_data

Dataset MNIST
    Number of datapoints: 1000
    Root location: data
    Split: Test

In [21]:
data_details

{'n_channels': 1, 'h_in': 28, 'w_in': 28, 'scale': 255.0}

In [22]:
DATA_DIM = data_details['n_channels']*data_details['h_in']*data_details['w_in']

X = []
Y = []

# Pytorch normalizes tensors (so need manual here!)
if args.use_test:
    for (x,y,_, _, _) in test_data:
        X.append(x/255.)
        Y.append(y)
else:
    for (x,y,_, _, _) in train_data:
        X.append(x/255.)
        Y.append(y)

X = np.array(X)
Y = np.array(Y)

num_samples = int(len(X)/2)
print(num_samples)

500


In [23]:
X_c1 = X[:num_samples].reshape(num_samples, DATA_DIM)
X_c2 = X[num_samples:].reshape(num_samples, DATA_DIM)

class_1 = args.class_1
class_2 = args.class_2

if not os.path.exists('distances'):
    os.makedirs('distances')

if not os.path.exists('cost_results'):
    os.makedirs('cost_results')

if args.use_full:
    subsample_sizes = [args.num_samples]
else:
    subsample_sizes = [args.num_samples]
   

rng = np.random.default_rng(77)

In [24]:
subsample_sizes

[500]

In [25]:
eps=[0,1,2,3,4,5,6,7,8,9,10]

In [26]:
loss_final=[]
for subsample_size in subsample_sizes:
  
    
    loss_list = []
    time_list = []
    num_edges_list = []

    if args.run_generic:
        time_generic_list = []

    if subsample_size == args.num_samples:
        num_reps=1
    else:
        num_reps=args.num_reps
    for rep in range(num_reps):
        indices_1 = rng.integers(num_samples,size=subsample_size)
        indices_2 = rng.integers(num_samples, size=subsample_size)

        if args.use_full:
            X_c1_curr = X_c1
            X_c2_curr = X_c2
        else:
            X_c1_curr = X_c1[indices_1]
            X_c2_curr = X_c2[indices_2]

        if args.use_test:
            dist_mat_name = args.dataset_in + '_test_' + str(class_1) + '_' + str(class_2) + '_' + str(subsample_size) + '_' + args.norm + '_rep' + str(rep) + '.npy'
        else:
            dist_mat_name = args.dataset_in + '_' + str(class_1) + '_' + str(class_2) + '_' + str(subsample_size) + '_' + args.norm + '_rep' + str(rep) + '.npy'
        if os.path.exists(dist_mat_name):
            print('Loading distances')
            D_12 = np.load('distances/' + dist_mat_name)
        else:
            if args.norm == 'l2':
                D_12 = scipy.spatial.distance.cdist(X_c1_curr,X_c2_curr,metric='euclidean')
            elif args.norm == 'linf':
                D_12 = scipy.spatial.distance.cdist(X_c1_curr,X_c2_curr,metric='chebyshev')
            np.save('distances/' + dist_mat_name, D_12)

        for m in range(len(eps)):
         # Add edge if cost 0
         edge_matrix = D_12 <= 2*eps[m]
         edge_matrix = edge_matrix.astype(float)

         num_edges = len(np.where(edge_matrix!=0)[0])
         print(num_edges,eps[m])
         num_edges_list.append(num_edges)

         n_1=subsample_size
         n_2=subsample_size
         weights=np.ones(n_1+n_2)

         # Create graph representation
         graph_rep_array = create_graph_rep(edge_matrix,n_1,n_2)

         time1= time.clock()
         q = queue.Queue()
         # Initial graph indices
         q.put(np.arange(n_1+n_2+2))
         sink_idx=n_1+n_2+1
         count=0
         classifier_probs=np.zeros(((n_1)+(n_2),2))
         while not q.empty():
            print('Current queue size at eps %s is %s' % (eps[m],q.qsize()))
            curr_idx_list=q.get()
            # print(q.qsize())
            list_1, list_2, flow_curr=find_flow_and_split(curr_idx_list,weights)
            # print(list_1,list_2,flow_curr.flow_value)
            if list_1 is not None:
                q.put(list_1)
            if list_2 is not None:
                q.put(list_2)
         time2 = time.clock()

         if args.run_generic:
            v=(n_1+n_2)
            num_edges=len(np.where(edge_matrix==1)[0])
            edges=np.where(edge_matrix==1)
            incidence_matrix=np.zeros((num_edges,v))

            for i in range(num_edges):
                j1=edges[0][i]
                j2=edges[1][i]+(n_1-1)
                incidence_matrix[i,j1]=1
                incidence_matrix[i,j2]=1

            G_in=np.vstack((incidence_matrix,np.eye(v)))
            h_in=np.ones((num_edges+v,1))
            p=(1.0/v)*np.ones((v,1))

            G_in_sparse_np=coo_matrix(G_in)

            G_in_sparse=spmatrix(1.0,G_in_sparse_np.nonzero()[0],G_in_sparse_np.nonzero()[1])

            solvers.options['maxiters']=1000

            time3=time.clock()
            output=minll(G_in_sparse,matrix(h_in),matrix(p))
            print(output['primal objective'])
            time4=time.clock()
            if output['status'] == 'optimal':
                time_generic_list.append(time4-time3)
            else:
                time_generic_list.append(-1.0*(time4-time3))

         loss = 0.0
         for i in range(len(classifier_probs)):
            if i<n_1:
                loss+=np.log(classifier_probs[i][0])
            elif i>=n_1:
                loss+=np.log(classifier_probs[i][1])
         loss=-1*loss/len(classifier_probs)
         print('Log loss for eps %s is %s' % (eps[m],loss))
         loss_final.append([eps[m],loss,subsample_size])
         loss_list.append(loss)
         time_list.append(time2-time1)

    loss_avg=np.mean(loss_list)
    loss_var=np.var(loss_list)
    time_avg=np.mean(time_list)
    time_var=np.var(time_list)
    num_edges_avg=np.mean(num_edges_list)

   


0 0


c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Current queue size at eps 0 is 1
Log loss for eps 0 is -0.0
0 1
Current queue size at eps 1 is 1
Log loss for eps 1 is -0.0
0 2
Current queue size at eps 2 is 1
Log loss for eps 2 is -0.0
86 3
Current queue size at eps 3 is 1
Current queue size at eps 3 is 2
Current queue size at eps 3 is 3
Current queue size at eps 3 is 4
Current queue size at eps 3 is 3
Current queue size at eps 3 is 2
Current queue size at eps 3 is 3
Current queue size at eps 3 is 2
Current queue size at eps 3 is 3
Current queue size at eps 3 is 4
Current queue size at eps 3 is 5
Current queue size at eps 3 is 6
Current queue size at eps 3 is 5
Current queue size at eps 3 is 4
Current queue size at eps 3 is 3
Current queue size at eps 3 is 2
Current queue size at eps 3 is 1
Log loss for eps 3 is 0.02257493658474787
4483 4
Current queue size at eps 4 is 1
Current queue size at eps 4 is 2
Current queue size at eps 4 is 3
Current queue size at eps 4 is 4
Current queue size at eps 4 is 3
Current queue size at eps 4 is 4

In [24]:
 edge_matrix.shape

(2500, 2500)

In [25]:
loss_final

[[0, -0.0, 2500],
 [1, -0.0, 2500],
 [2, 0.0005004024235381879, 2500],
 [3, 0.032377251425240644, 2500],
 [4, 0.3458605759331595, 2500],
 [5, 0.6780286674263256, 2500],
 [6, 0.6931472005678916, 2500],
 [7, 0.6931472005678916, 2500],
 [8, 0.6931472005678916, 2500],
 [9, 0.6931472005678916, 2500],
 [10, 0.6931472005678916, 2500]]

In [24]:
num_edges_avg

36205.375

In [25]:
edge_matrix

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
classifier_probs

array([[0.47582873, 0.52417127],
       [0.47582873, 0.52417127],
       [0.75      , 0.25      ],
       ...,
       [0.47582873, 0.52417127],
       [0.47582873, 0.52417127],
       [0.        , 1.        ]])

In [27]:
len(classifier_probs)

1600

In [27]:
D_12

array([[10.84659166, 11.05460773, 11.2006371 , ...,  9.84820743,
        10.14525417, 10.79192316],
       [10.67833272,  9.94280066,  9.73412725, ...,  8.24689659,
         8.65168788,  8.87287206],
       [12.37230167, 11.21644177, 11.46179988, ..., 10.37937653,
        11.40822194, 11.02153078],
       ...,
       [ 9.76150589,  9.54798964,  8.89217567, ...,  6.50025759,
         7.60278283,  8.02944217],
       [ 9.98775567,  9.52234467,  9.70566369, ...,  7.67758312,
         8.55505094,  9.23188147],
       [11.19455228,  9.94081061, 10.76381081, ...,  9.55363982,
        10.58924977, 10.04681698]])